In [46]:
import pandas as pd
import json
import requests
import os
import openpyxl
import time

from helpers import *

## get date from API

In [52]:
# display the API's uber-helful X-Status-Reason in the unlikely event 
# that we make a bad request
def handle_request_error(response, query_url, api_key):
    if response.status_code == 400:
        x_header_value = response.headers.get("X-Status-Reason")
        if x_header_value:
            print(f"Error: {x_header_value}")
        else:
            print("400 Bad Request: No X-Status-Reason found")
        exit(1)
    elif response.status_code == 429:
        print("Throttled response from the api, retrying in {} seconds".format(response.headers["Retry-After"]))
        time.sleep(int(response.headers["Retry-After"]))  # Number of seconds to wait before sending next request
        response = requests.get(query_url, headers={"X-Api-Key": api_key})
        return response
    else:
        response.raise_for_status()
    return response

In [38]:
with open('keys/patsviews_key', 'r') as file:
    api_key = file.read().strip()

In [39]:
base_url = 'https://search.patentsview.org' # this is the same for all endpoints
endpoint = 'api/v1/patent'

In [55]:
names = ["FW", "LC", "LIB",  "SC", "SPV", "Wind"]  # technology file names
# names = ["Wind"]  # technology file names"Wind"

In [41]:
all_dfs = []
for n in names:
    df = pd.read_excel(f'unlabeled_data/DB_{n}_USPTO.xlsx')
    all_dfs.append(df)

In [53]:
def make_get_request(endpoint, param_dict):
    param_string = "&".join([f"{param_name}={json.dumps(param_val)}" for param_name, param_val in param_dict.items()])
    # note the json.dumps to ensure strings are surrounded by double quotes instead of single
    # the api will not accept single quotes
    query_url = f"{base_url}/{endpoint.strip('/')}/?{param_string}"
    response = requests.get(query_url, headers={"X-Api-Key": api_key})
    response = handle_request_error(response, query_url, api_key)
    return response

In [54]:
i=0
for df in all_dfs:
    print('starting date extraction for', names[i])
    
    df.dropna(subset=['Publn_Nr'], inplace=True)  # drop non defined publication numbers
    
    pub_numbers = df['Publn_Nr'].unique().astype(list)
    df_u = df.drop_duplicates(subset=['Publn_Nr'])  # unique claims (sort by patent)
    pub_numbers = pub_numbers.astype(int).astype(str)

    split_pubs = [pub_numbers[i:i+100] for i in range(0, len(pub_numbers), 100)]  # split into chunks of 100 cause of API limits

    # run the API request
    list_of_dfs = []
    for pub_nrs in split_pubs:
        param_dict = {
            "f" : ["patent_id", "patent_date"],  # get patent id and date
            "o" : {"size":200}, 
            "q" : {"patent_id": list(pub_nrs)},  # insert publication id
            # "s" : [{"patent_date":"asc"}], 
        }
        response = make_get_request(endpoint, param_dict)
        response_unpacked = pd.DataFrame(response.json()[response_key(endpoint)])
        list_of_dfs.append(response_unpacked)

    df_dates = pd.concat(list_of_dfs)
    df_dates["patent_date"] = pd.to_datetime(df_dates["patent_date"], format="%Y-%m-%d")

    # save to file
    n = names[i]
    df_dates.to_csv(f'unlabeled_data/DB_{n}_USPTO_dates.csv', index=False)
    print('saved file for date extraction for', n)
    i += 1

starting date extraction for FW
Throttled response from the api, retrying in 15 seconds
Throttled response from the api, retrying in 1 seconds
Throttled response from the api, retrying in 15 seconds
saved file for date extraction for FW
starting date extraction for LC
Throttled response from the api, retrying in 16 seconds
Throttled response from the api, retrying in 17 seconds
saved file for date extraction for LC
starting date extraction for LIB
Throttled response from the api, retrying in 15 seconds
Throttled response from the api, retrying in 16 seconds
Throttled response from the api, retrying in 17 seconds
Throttled response from the api, retrying in 15 seconds
Throttled response from the api, retrying in 18 seconds
Throttled response from the api, retrying in 16 seconds
Throttled response from the api, retrying in 14 seconds
Throttled response from the api, retrying in 14 seconds
Throttled response from the api, retrying in 1 seconds
Throttled response from the api, retrying in 

## merge date into unlabled data set

first sort data sets according to id number

In [77]:
names = ["FW", "LC", "LIB",  "SC", "SPV", "Wind"]  # technology file names

In [78]:
for n in names:
    # load df
    df_dates = pd.read_csv(f'unlabeled_data/DB_{n}_USPTO_dates.csv')
    df = pd.read_excel(f'unlabeled_data/DB_{n}_USPTO.xlsx')

    # moodify and sort
    df.dropna(subset=['Publn_Nr'], inplace=True)  # drop non defined publication numbers
    df['Publn_Nr'] = df['Publn_Nr'].astype(int)   # convert to int
    df.sort_values(by='Publn_Nr', inplace=True)   # sort by patent number
    df['Publn_Date'] = None  # add empty column for dates
    df_dates['patent_id'] = df_dates['patent_id'].astype(int)  # convert to int and then to string
    df_dates.sort_values(by='patent_id', inplace=True)  # sort by patent number

    # iterate over the rows and insert the dates
    for idx, row in df_dates.iterrows():
        df.loc[df['Publn_Nr'] == row['patent_id'], 'Publn_Date'] = row['patent_date']
    
    # sort by date
    df.sort_values(by='Publn_Date', inplace=True)

    # save to file
    df.to_excel(f'sorted_data/DB_{n}_USPTO_sorted.xlsx', index=False)